In [2]:
import numpy as np
import matplotlib.pyplot as plt
from crccheck.crc import Crc24Ble

In [3]:
ADV_IND         = 0x00
ADV_SCAN_IND    = 0x03
ADV_NONCONN_IND = 0x04
ADV_DIRECT_IND  = 0x08

def scrambling_code(channel, length):
    lfsr = np.unpackbits(np.array([channel], dtype=np.ubyte))
    lfsr[1] = 1
    lfsr = lfsr[1:]
    out = np.zeros(length * 8, dtype=np.ubyte)
    for i in range(length * 8):
        out[i] = lfsr[6]
        lfsr[6] = lfsr[5]
        lfsr[5] = lfsr[4]
        lfsr[4] = out[i] ^ lfsr[3]
        lfsr[3] = lfsr[2]
        lfsr[2] = lfsr[1]
        lfsr[1] = lfsr[0]
        lfsr[0] = out[i]
    return np.packbits(out, bitorder="little")

def gen_ad_pdu(ad_type, access_address, payload, tx_add=0, rx_add=0):
    payload_buf = b""
    flag = (rx_add << 3) + (tx_add << 2)
    for data_type in payload:
        section = bytes([len(payload[data_type]) + 1, data_type]) + payload[data_type]
        payload_buf += section
    header = bytes([ad_type + (flag << 4), len(payload_buf) + len(access_address)])
    return header + access_address + payload_buf

def gen_ble_pkt(access_address, pdu, whitening=False):
    if access_address[0] & 1 == 1:
        preamble = b"\x55"
    else:
        preamble = b"\xaa"
    crc = Crc24Ble.calcbytes(pdu)[::-1]
    pdu_crc = pdu + crc
    if whitening:
        pdu_crc = np.frombuffer(pdu_crc, dtype=np.ubyte) ^ scrambling_code(38 ,len(pdu_crc))
        pdu_crc = pdu_crc.tobytes()
    return preamble + access_address + pdu_crc

def hexdump(bytes_input, width=16):
    current = 0
    end = len(bytes_input)
    result = ""
    while current < end:
        result += "%08x: " % current
        byte_slice = bytes_input[current : current + width]
        for b in byte_slice:
            result += "%02x " % b
        for _ in range(width - len(byte_slice)):
            result += " " * 3
        result += " "
        for b in byte_slice:
            if (b >= 32) and (b < 127):
                result += chr(b)
            else:
                result += "."
        result += "\n"
        current += width
    return result

In [8]:
uuid = b"\x01\x12\x23\x34\x45\x56\x67\x78\x89\x9a\xab\xbc\xcd\xde\xef\xf0"

payload = {
    0x01: b"\x06",
    0xff: b"\x59\x00\x02\x15" + uuid + b"\x00\x00\x00\x00\xc1",
    0x09: "Test iBeacon".encode(),
}

pdu = gen_ad_pdu(ADV_IND, b"\x05\xa2\x17\x6e\x3d\x71", payload, tx_add=1)
print(hexdump(pdu))
pkt = gen_ble_pkt(b"\xd6\xbe\x89\x8e", pdu, whitening=True)
print(hexdump(pkt))

with open("ibeacon.dat", "wb") as f:
    tmp = np.frombuffer(pkt, dtype=np.ubyte)
    f.write(tmp.tobytes())

00000000: 40 32 05 a2 17 6e 3d 71 02 01 06 1a ff 59 00 02  @2...n=q.....Y..
00000010: 15 01 12 23 34 45 56 67 78 89 9a ab bc cd de ef  ...#4EVgx.......
00000020: f0 00 00 00 00 c1 0d 09 54 65 73 74 20 69 42 65  ........Test iBe
00000030: 61 63 6f 6e                                      acon

00000000: aa d6 be 89 8e 40 32 05 a2 17 6e 3d 71 02 01 06  .....@2...n=q...
00000010: 1a ff 59 00 02 15 01 12 23 34 45 56 67 78 89 9a  ..Y.....#4EVgx..
00000020: ab bc cd de ef f0 00 00 00 00 c1 0d 09 54 65 73  .............Tes
00000030: 74 20 69 42 65 61 63 6f 6e 3f fa 68              t iBeacon?.h

